In [2]:
import io
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import mmap
import numpy
import soundfile
import torchaudio
import torch
import os
import re
import pandas as pd
from datasets import load_dataset
import ffmpeg

from collections import defaultdict
from IPython.display import Audio, display
from pathlib import Path
from pydub import AudioSegment

from seamless_communication.inference import Translator
from seamless_communication.streaming.dataloaders.s2tt import SileroVADSilenceRemover
import warnings
warnings.filterwarnings("ignore")

In [3]:
from datasets import load_dataset
from seamless_communication.inference import Translator
from jiwer import wer
from sacrebleu import corpus_bleu

In [4]:
# Initialize a Translator object with a multitask model, vocoder on the GPU.

model_name = "seamlessM4T_v2_large"
vocoder_name = "vocoder_v2" if model_name == "seamlessM4T_v2_large" else "vocoder_36langs"

translator = Translator(
    model_name,
    vocoder_name,
    device=torch.device("cuda"), # Changed from "cuda:0" to "cpu"
    dtype=torch.float16,
)

Using the cached checkpoint of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached tokenizer of seamlessM4T_v2_large. Set `force` to `True` to download again.
Using the cached checkpoint of vocoder_v2. Set `force` to `True` to download again.


In [5]:
from datasets import load_dataset
import torch
import torchaudio
def resample_to_16k(audio, orig_sr):
    # implement resampling logic here, e.g. torchaudio.transforms.Resample
    return torchaudio.transforms.Resample(orig_sr, 16000)(torch.tensor(audio)).numpy()

In [6]:
import whisper

# Load Whisper-Large once (outside function, so it’s not reloaded every call)
whisper_model = whisper.load_model("large-v3", device="cuda")

whisper.audio.FFMPEG_PATH = "/home/aj/Bhavna/ffmpeg_bin/ffmpeg"

In [7]:
#%pip install seamless_communication

from seamless_communication.inference import SequenceGeneratorOptions
text_opts = SequenceGeneratorOptions(
    beam_size=5
    #temperature=0.7,   # 0 for deterministic, >0 adds diversity
)

# Beam search for unit hypotheses
unit_opts = SequenceGeneratorOptions(
    beam_size=5
)

In [8]:
import string

def normalize(text):
    return text.lower().translate(str.maketrans("", "", string.punctuation)).strip()

In [9]:
def save_dataframe(df, lang, out_dir="/scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs"):
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"{lang}_results.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved results to {out_path}")

In [10]:
from indicnlp.tokenize import indic_tokenize

In [11]:
def normalize(text):
    return text.lower().translate(str.maketrans("", "", string.punctuation)).strip()

In [12]:
def apply_tokenizer(texts, lang_code):
    """Apply IndicNLP/Urdu tokenization for En→Indic evaluation"""
    if not texts:
        return texts
    elif lang_code in ["hi", "bn", "te", "ta", "ml", "kn", "gu", "mr", "pa", "or"]:
        return [" ".join(indic_tokenize.trivial_tokenize(t, lang=lang_code)) for t in texts]
    else:  # fallback (English etc.)
        return texts

In [ ]:
def run_translation_for_language(sm4t_src_lang,fleurs_src_lang,sm4t_tgt_lang, fleurs_tgt_lang, full_tasks=True):
    """
    Run translation pipeline for one language.
    full_tasks=True → Run all 4 tasks 
    full_tasks=False → Run only S2TT, T2TT
    """
    print("\n" + "="*60)
    print(f"🔹 Processing Target Language: {sm4t_tgt_lang.upper()} ({fleurs_tgt_lang})")
    print("="*60)

    # Load datasets
    src_dataset = load_dataset("google/fleurs", fleurs_src_lang, split="test")
    tgt_dataset = load_dataset("google/fleurs", fleurs_tgt_lang, split="test")

    src_by_id = {item["id"]: item for item in src_dataset}
    tgt_by_id = {item["id"]: item for item in tgt_dataset}
    common_ids = sorted(set(src_by_id.keys()) & set(tgt_by_id.keys()))

    print(f"Found {len(common_ids)} parallel sentences")

    references, hypotheses_s2tt, hypotheses_t2tt = [], [], []
    predicted_s2s, predicted_t2s = [], []
    source_texts = []

    for sentence_id in common_ids:
        src = src_by_id[sentence_id]
        tgt = tgt_by_id[sentence_id]

        src_audio = src["audio"]["array"]
        src_sr = src["audio"]["sampling_rate"]
        src_text = src["transcription"]
        tgt_text = tgt["transcription"]

        references.append([tgt_text])
        source_texts.append(src_text)

        if src_sr != 16000:
            src_audio = resample_to_16k(src_audio, src_sr)

        base_dir = "/scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/input_audios_of_eng"
        lang_dir = os.path.join(base_dir, sm4t_tgt_lang)
        os.makedirs(lang_dir, exist_ok=True)
        audio_path = os.path.join(lang_dir, f"input_{sm4t_tgt_lang}_{sentence_id}.wav")

        torchaudio.save(audio_path, torch.tensor(src_audio).unsqueeze(0), 16000)

        # --- S2TT ---
        s2tt_out, _ = translator.predict(
            input=audio_path, task_str="s2tt", tgt_lang=sm4t_tgt_lang,text_generation_opts=text_opts
        )
        hypotheses_s2tt.append(normalize(str(s2tt_out[0])))

        # --- T2TT ---
        t2tt_out, _ = translator.predict(
            input=src_text, task_str="t2tt", src_lang=sm4t_src_lang, tgt_lang=sm4t_tgt_lang,text_generation_opts=text_opts
        )
        hypotheses_t2tt.append(normalize(str(t2tt_out[0])))

        if full_tasks:
            # --- S2ST + Whisper ASR ---
            _, s2s_audio_out = translator.predict(
                input=audio_path, task_str="s2st", tgt_lang=sm4t_tgt_lang,text_generation_opts=text_opts,unit_generation_opts=unit_opts
            )
            base_dir = "/scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/s2s_outputs"
            lang_dir = os.path.join(base_dir, sm4t_tgt_lang)
            os.makedirs(lang_dir, exist_ok=True)
            s2s_path= os.path.join(lang_dir, f"s2s_{sm4t_tgt_lang}_{sentence_id}.wav")

            torchaudio.save(
                s2s_path,
                s2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
                s2s_audio_out.sample_rate,
            )

            # Use Whisper-Large for ASR
            s2s_result = whisper_model.transcribe(
                audio=s2s_path,
                task="transcribe",
                temperature=0.0,   # greedy, deterministic
                beam_size=None
            )
            predicted_s2s.append(normalize(s2s_result["text"]))

            # --- T2ST + Whisper ASR ---
            _, t2s_audio_out = translator.predict(
                input=src_text, task_str="t2st", src_lang=sm4t_src_lang, tgt_lang=sm4t_tgt_lang,text_generation_opts=text_opts,unit_generation_opts=unit_opts
            )
            base_dir = "/scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/t2s_outputs"
            lang_dir = os.path.join(base_dir, sm4t_tgt_lang)
            os.makedirs(lang_dir, exist_ok=True)
            t2s_path = os.path.join(lang_dir, f"t2s_{sm4t_tgt_lang}_{sentence_id}.wav")

            torchaudio.save(
                t2s_path,
                t2s_audio_out.audio_wavs[0][0].to(torch.float32).cpu(),
                t2s_audio_out.sample_rate,
            )

            # Use Whisper-Large for ASR
            t2s_result = whisper_model.transcribe(
                audio=t2s_path,
                task="transcribe",
                temperature=0.0,
                beam_size=None
            )
            predicted_t2s.append(normalize(t2s_result["text"]))

    # Build dataframe
    data = {
        "source_text": source_texts,
        "reference_text": [r[0] for r in references],
        "S2TT_prediction": hypotheses_s2tt,
        "T2TT_prediction": hypotheses_t2tt
    }
    if full_tasks:
        data["S2ST_ASR"] = predicted_s2s
        data["T2ST_ASR"] = predicted_t2s

    df = pd.DataFrame(data)

    save_dataframe(df, sm4t_tgt_lang)

    return references, hypotheses_s2tt, hypotheses_t2tt, predicted_s2s, predicted_t2s

In [14]:
from sacrebleu import corpus_chrf

def compute_metrics(src_lang, tgt_lang, references, hypotheses_s2tt, hypotheses_t2tt, predicted_s2s, predicted_t2s):
    print(f"\nComputing metrics for {src_lang.upper()} → {tgt_lang.upper()}")

    # Normalize hyps
    hypotheses_s2tt = [str(h) for h in hypotheses_s2tt]
    hypotheses_t2tt = [str(h) for h in hypotheses_t2tt]
    predicted_s2s   = [str(h) for h in predicted_s2s]
    predicted_t2s   = [str(h) for h in predicted_t2s]

    # Normalize refs
    references_norm = [[str(r) for r in refset] for refset in references]
    multi_references = list(zip(*references_norm))

    # ---- Tokenization switch ----
    if src_lang == "en":  
        # Eng → Indic → tokenize target
        hypotheses_s2tt = apply_tokenizer(hypotheses_s2tt, tgt_lang)
        hypotheses_t2tt = apply_tokenizer(hypotheses_t2tt, tgt_lang)
        predicted_s2s   = apply_tokenizer(predicted_s2s, tgt_lang)
        predicted_t2s   = apply_tokenizer(predicted_t2s, tgt_lang)
        tokenized_refs = [apply_tokenizer(refs, tgt_lang) for refs in multi_references]
    else:
        # Indic → En → use sacreBLEU default tokenizer
        tokenized_refs = multi_references

    tokenized_refs = list(zip(*tokenized_refs))

    metrics = {}

    # --------------------
    # S2TT
    # --------------------
    metrics["S2TT_BLEU"]      = corpus_bleu(hypotheses_s2tt, tokenized_refs).score
    metrics["S2TT_chrF++"]    = corpus_chrf(hypotheses_s2tt, tokenized_refs).score
    metrics["S2TT_chrF2++"]   = corpus_chrf(hypotheses_s2tt, tokenized_refs, beta=2).score
    metrics["S2TT_WER"]       = sum(wer(ref[0], hyp) for ref, hyp in zip(references_norm, hypotheses_s2tt)) / len(references_norm)

    # --------------------
    # T2TT
    # --------------------
    metrics["T2TT_BLEU"]      = corpus_bleu(hypotheses_t2tt, tokenized_refs).score
    metrics["T2TT_chrF++"]    = corpus_chrf(hypotheses_t2tt, tokenized_refs).score
    metrics["T2TT_chrF2++"]   = corpus_chrf(hypotheses_t2tt, tokenized_refs, beta=2).score
    metrics["T2TT_WER"]       = sum(wer(ref[0], hyp) for ref, hyp in zip(references_norm, hypotheses_t2tt)) / len(references_norm)

    # --------------------
    # S2ST
    # --------------------
    if predicted_s2s:
        metrics["S2ST_BLEU"]    = corpus_bleu(predicted_s2s, tokenized_refs).score
        metrics["S2ST_chrF++"]  = corpus_chrf(predicted_s2s, tokenized_refs).score
        metrics["S2ST_chrF2++"] = corpus_chrf(predicted_s2s, tokenized_refs, beta=2).score
        metrics["S2ST_WER"]     = sum(wer(ref[0], hyp) for ref, hyp in zip(references_norm, predicted_s2s)) / len(references_norm)

    # --------------------
    # T2ST
    # --------------------
    if predicted_t2s:
        metrics["T2ST_BLEU"]    = corpus_bleu(predicted_t2s, tokenized_refs).score
        metrics["T2ST_chrF++"]  = corpus_chrf(predicted_t2s, tokenized_refs).score
        metrics["T2ST_chrF2++"] = corpus_chrf(predicted_t2s, tokenized_refs, beta=2).score
        metrics["T2ST_WER"]     = sum(wer(ref[0], hyp) for ref, hyp in zip(references_norm, predicted_t2s)) / len(references_norm)

    return metrics


In [15]:
refs1, hyps_s2tt1, hyps_t2tt1, preds_s2s1, preds_t2s1 = run_translation_for_language("eng", "en_us", "hin", "hi_in", full_tasks=True)



🔹 Processing Target Language: HIN (hi_in)
Found 265 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/hin_results.csv


In [16]:
refs2, hyps_s2tt2, hyps_t2tt2, preds_s2s2, preds_t2s2 = run_translation_for_language("eng", "en_us", "ben", "bn_in", full_tasks=True)


🔹 Processing Target Language: BEN (bn_in)
Found 349 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/ben_results.csv


In [17]:
refs3, hyps_s2tt3, hyps_t2tt3, preds_s2s3, preds_t2s3 = run_translation_for_language("eng", "en_us", "tel", "te_in", full_tasks=True)


🔹 Processing Target Language: TEL (te_in)
Found 302 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/tel_results.csv


In [45]:
refs4, hyps_s2tt4, hyps_t2tt4, preds_s2s4, preds_t2s4 = run_translation_for_language("eng", "en_us", "tam", "ta_in", full_tasks=False)


🔹 Processing Target Language: TAM (ta_in)
Found 336 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/tam_results.csv


In [19]:
refs5, hyps_s2tt5, hyps_t2tt5, preds_s2s5, preds_t2s5 = run_translation_for_language("eng", "en_us", "mal", "ml_in", full_tasks=False)


🔹 Processing Target Language: MAL (ml_in)
Found 344 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/mal_results.csv


In [20]:
refs6, hyps_s2tt6, hyps_t2tt6, preds_s2s6, preds_t2s6 = run_translation_for_language("eng", "en_us", "kan", "kn_in", full_tasks=False)


🔹 Processing Target Language: KAN (kn_in)
Found 344 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/kan_results.csv


In [21]:
refs7, hyps_s2tt7, hyps_t2tt7, preds_s2s7, preds_t2s7 = run_translation_for_language("eng", "en_us", "guj", "gu_in", full_tasks=False)


🔹 Processing Target Language: GUJ (gu_in)
Found 349 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/guj_results.csv


In [22]:
refs8, hyps_s2tt8, hyps_t2tt8, preds_s2s8, preds_t2s8 = run_translation_for_language("eng", "en_us", "mar", "mr_in", full_tasks=False)


🔹 Processing Target Language: MAR (mr_in)
Found 349 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/mar_results.csv


In [ ]:
refs9, hyps_s2tt9, hyps_t2tt9, preds_s2s9, preds_t2s9 = run_translation_for_language("eng", "en_us", "pan", "pa_in", full_tasks=False)


🔹 Processing Target Language: PAN (pa_in)
Found 279 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/pan_results.csv


: 

In [25]:
refs10, hyps_s2tt10, hyps_t2tt10, preds_s2s10, preds_t2s10 = run_translation_for_language("eng", "en_us", "ory", "or_in", full_tasks=False)


🔹 Processing Target Language: ORY (or_in)
Found 334 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/ory_results.csv


In [26]:
refs11, hyps_s2tt11, hyps_t2tt11, preds_s2s11, preds_t2s11 = run_translation_for_language("eng", "en_us", "asm", "as_in", full_tasks=False)


🔹 Processing Target Language: ASM (as_in)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Found 349 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/asm_results.csv


In [29]:
refs12, hyps_s2tt12, hyps_t2tt12, preds_s2s12, preds_t2s12 = run_translation_for_language("eng", "en_us", "npi","ne_np", full_tasks=False)


🔹 Processing Target Language: NPI (ne_np)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Found 343 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/npi_results.csv


In [28]:
refs13, hyps_s2tt13, hyps_t2tt13, preds_s2s13, preds_t2s13 = run_translation_for_language("eng", "en_us", "snd", "sd_in", full_tasks=False)


🔹 Processing Target Language: SND (sd_in)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Found 350 parallel sentences
Saved results to /scratch/aj/Bhavna/bhav_venv_311/f-Eng-X-Direct/f-Eng-X-Direct-CSVs/snd_results.csv


In [30]:
compute_metrics("en", "hi_in", refs1, hyps_s2tt1, hyps_t2tt1, preds_s2s1, preds_t2s1)


Computing metrics for EN → HI_IN


{'S2TT_BLEU': 47.20758038942707,
 'S2TT_chrF++': 67.1422707430045,
 'S2TT_chrF2++': 67.1422707430045,
 'S2TT_WER': 0.5858850193121427,
 'T2TT_BLEU': 53.05903666964891,
 'T2TT_chrF++': 73.29101641233864,
 'T2TT_chrF2++': 73.29101641233864,
 'T2TT_WER': 0.5608586884988706,
 'S2ST_BLEU': 20.078866641712107,
 'S2ST_chrF++': 59.323928226195285,
 'S2ST_chrF2++': 59.323928226195285,
 'S2ST_WER': 0.6807390948093932,
 'T2ST_BLEU': 20.2292806480005,
 'T2ST_chrF++': 60.11316098674055,
 'T2ST_chrF2++': 60.11316098674055,
 'T2ST_WER': 0.6835818748195117}

In [31]:
compute_metrics("en", "bn_in", refs2, hyps_s2tt2, hyps_t2tt2, preds_s2s2, preds_t2s2)


Computing metrics for EN → BN_IN


{'S2TT_BLEU': 20.51174794065995,
 'S2TT_chrF++': 46.573264080995735,
 'S2TT_chrF2++': 46.573264080995735,
 'S2TT_WER': 0.7176131372989533,
 'T2TT_BLEU': 13.717032573436434,
 'T2TT_chrF++': 46.96624079209037,
 'T2TT_chrF2++': 46.96624079209037,
 'T2TT_WER': 0.7319149032788587,
 'S2ST_BLEU': 3.7199102928113716,
 'S2ST_chrF++': 37.033558176436685,
 'S2ST_chrF2++': 37.033558176436685,
 'S2ST_WER': 0.9165609893600437,
 'T2ST_BLEU': 3.7199102928113716,
 'T2ST_chrF++': 38.61902695277767,
 'T2ST_chrF2++': 38.61902695277767,
 'T2ST_WER': 0.9288998385967467}

In [32]:
compute_metrics("en","te_in", refs3, hyps_s2tt3, hyps_t2tt3, preds_s2s3, preds_t2s3)


Computing metrics for EN → TE_IN


{'S2TT_BLEU': 22.229849552064017,
 'S2TT_chrF++': 65.36304413445346,
 'S2TT_chrF2++': 65.36304413445346,
 'S2TT_WER': 0.6862416954139396,
 'T2TT_BLEU': 22.765893232556483,
 'T2TT_chrF++': 67.76201712142644,
 'T2TT_chrF2++': 67.76201712142644,
 'T2TT_WER': 0.6990720558165725,
 'S2ST_BLEU': 3.4585921141027356,
 'S2ST_chrF++': 31.203694604442983,
 'S2ST_chrF2++': 31.203694604442983,
 'S2ST_WER': 0.9101295785795336,
 'T2ST_BLEU': 3.4585921141027356,
 'T2ST_chrF++': 28.79424982029214,
 'T2ST_chrF2++': 28.79424982029214,
 'T2ST_WER': 0.915643101194562}

In [33]:
compute_metrics("en","ta_in", refs4, hyps_s2tt4, hyps_t2tt4, preds_s2s4, preds_t2s4)


Computing metrics for EN → TA_IN


{'S2TT_BLEU': 9.83748990557782,
 'S2TT_chrF++': 53.53349693501887,
 'S2TT_chrF2++': 53.53349693501887,
 'S2TT_WER': 0.7242391794911189,
 'T2TT_BLEU': 25.400289715190983,
 'T2TT_chrF++': 59.54718891638381,
 'T2TT_chrF2++': 59.54718891638381,
 'T2TT_WER': 0.7244738057602584}

In [34]:
compute_metrics("en","ml_in", refs5, hyps_s2tt5, hyps_t2tt5, preds_s2s5, preds_t2s5)


Computing metrics for EN → ML_IN


{'S2TT_BLEU': 20.098339913206324,
 'S2TT_chrF++': 63.73427954208568,
 'S2TT_chrF2++': 63.73427954208568,
 'S2TT_WER': 0.7703561164210542,
 'T2TT_BLEU': 8.516593018819643,
 'T2TT_chrF++': 56.71883960748831,
 'T2TT_chrF2++': 56.71883960748831,
 'T2TT_WER': 0.7887866641034194}

In [35]:
compute_metrics("en","kn_in", refs6, hyps_s2tt6, hyps_t2tt6, preds_s2s6, preds_t2s6)


Computing metrics for EN → KN_IN


{'S2TT_BLEU': 8.032276872815308,
 'S2TT_chrF++': 52.07141410336956,
 'S2TT_chrF2++': 52.07141410336956,
 'S2TT_WER': 0.7301583821932678,
 'T2TT_BLEU': 15.580105704117443,
 'T2TT_chrF++': 58.283865852094785,
 'T2TT_chrF2++': 58.283865852094785,
 'T2TT_WER': 0.731754491323938}

In [36]:
compute_metrics("en","gu_in", refs7, hyps_s2tt7, hyps_t2tt7, preds_s2s7, preds_t2s7)


Computing metrics for EN → GU_IN


{'S2TT_BLEU': 13.508625657351418,
 'S2TT_chrF++': 40.02976054989051,
 'S2TT_chrF2++': 40.02976054989051,
 'S2TT_WER': 0.6773370563588701,
 'T2TT_BLEU': 14.009047908905242,
 'T2TT_chrF++': 41.014062907013304,
 'T2TT_chrF2++': 41.014062907013304,
 'T2TT_WER': 0.6827688630383182}

In [37]:
compute_metrics("en","mr_in", refs8, hyps_s2tt8, hyps_t2tt8, preds_s2s8, preds_t2s8)


Computing metrics for EN → MR_IN


{'S2TT_BLEU': 15.727800941615351,
 'S2TT_chrF++': 51.991014342379735,
 'S2TT_chrF2++': 51.991014342379735,
 'S2TT_WER': 0.7115384370655192,
 'T2TT_BLEU': 14.503996062829852,
 'T2TT_chrF++': 54.806454755521806,
 'T2TT_chrF2++': 54.806454755521806,
 'T2TT_WER': 0.7520936178839361}

In [38]:
compute_metrics("en","pa_in", refs9, hyps_s2tt9, hyps_t2tt9, preds_s2s9, preds_t2s9)


Computing metrics for EN → PA_IN


{'S2TT_BLEU': 37.61370954911621,
 'S2TT_chrF++': 59.71624061185019,
 'S2TT_chrF2++': 59.71624061185019,
 'S2TT_WER': 0.6201570979038564,
 'T2TT_BLEU': 30.143352515082135,
 'T2TT_chrF++': 52.55582129095531,
 'T2TT_chrF2++': 52.55582129095531,
 'T2TT_WER': 0.657551606898665}

In [39]:
compute_metrics("en","or_in", refs10, hyps_s2tt10, hyps_t2tt10, preds_s2s10, preds_t2s10)


Computing metrics for EN → OR_IN


{'S2TT_BLEU': 8.90763711246724,
 'S2TT_chrF++': 52.73232032726691,
 'S2TT_chrF2++': 52.73232032726691,
 'S2TT_WER': 0.7450997409593849,
 'T2TT_BLEU': 7.655122720591221,
 'T2TT_chrF++': 44.52031608947945,
 'T2TT_chrF2++': 44.52031608947945,
 'T2TT_WER': 0.8251359523183592}

In [40]:
compute_metrics("en","as_in", refs11, hyps_s2tt11, hyps_t2tt11, preds_s2s11, preds_t2s11)


Computing metrics for EN → AS_IN


{'S2TT_BLEU': 4.348949343910558,
 'S2TT_chrF++': 46.43426595275811,
 'S2TT_chrF2++': 46.43426595275811,
 'S2TT_WER': 0.8195492533473043,
 'T2TT_BLEU': 8.156196366527345,
 'T2TT_chrF++': 54.05801710693592,
 'T2TT_chrF2++': 54.05801710693592,
 'T2TT_WER': 0.8654719353728063}

In [41]:
compute_metrics("en","ne_np", refs12, hyps_s2tt12, hyps_t2tt12, preds_s2s12, preds_t2s12)


Computing metrics for EN → NE_NP


{'S2TT_BLEU': 16.26170171519489,
 'S2TT_chrF++': 67.81577468420329,
 'S2TT_chrF2++': 67.81577468420329,
 'S2TT_WER': 0.731678590318758,
 'T2TT_BLEU': 14.247788801610149,
 'T2TT_chrF++': 78.63478822198536,
 'T2TT_chrF2++': 78.63478822198536,
 'T2TT_WER': 0.767599695602735}

In [42]:
compute_metrics("en","snd_in", refs13, hyps_s2tt13, hyps_t2tt13, preds_s2s13, preds_t2s13)


Computing metrics for EN → SND_IN


{'S2TT_BLEU': 22.813997135031524,
 'S2TT_chrF++': 43.94689607557326,
 'S2TT_chrF2++': 43.94689607557326,
 'S2TT_WER': 0.6855863957906705,
 'T2TT_BLEU': 21.951524426618445,
 'T2TT_chrF++': 45.04925955622581,
 'T2TT_chrF2++': 45.04925955622581,
 'T2TT_WER': 0.7175115345646228}